In [17]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

dataset = np.loadtxt("./features.txt", delimiter=" ");
print(dataset.shape)

X = dataset[:, :35]
Y = dataset[:, 35]
botnetDataset = dataset[dataset[:,35]==1, :]
normalDataset = dataset[dataset[:,35]==0, :]

botnetX = botnetDataset[:,:35]
botnetY = botnetDataset[:, 35]

normalX = normalDataset[:,:35]
normalY = normalDataset[:, 35]

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledBotnetX = scaler.fit_transform(botnetX)
rescaledNormalX = scaler.fit_transform(normalX)
rescaledX = scaler.fit_transform(X)

(52580, 36)


In [18]:
genericFeatures = range(6)
aggregateFeatures = range(6, 30)
subnetFeatures = range(30, 33)
periodicCommnFeatures = [33, 34]

withoutGenericFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in genericFeatures]
withoutAggregateFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in aggregateFeatures]
withoutSubnetFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in subnetFeatures]
withoutPeriodicCommnFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in periodicCommnFeatures]

def updateColumns(columns):
    global rescaledBotnetX, rescaledNormalX, rescaledX
    rescaledBotnetX = rescaledBotnetX[:, columns]
    rescaledNormalX = rescaledNormalX[:, columns]
    rescaledX = rescaledX[:, columns]
    
updateColumns(withoutSubnetFeatures)

In [19]:
def GetNextBotnetSet(rescaledX, Y):
    step = 4000
    prevStep = 0
    while step < rescaledX.shape[0]:
        yield rescaledX[prevStep:step, :], Y[prevStep:step]
        prevStep = step
        step = step + 4000
        
    return rescaledX[prevStep:, :], Y[prevStep:]

In [20]:
from sklearn.cross_validation import train_test_split

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model2 = RandomForestClassifier(n_estimators=100)
    model2 = model2.fit(X_train, Y_train)
    print(model2.score(X_test, Y_test))

0.998766193708
0.999383096854
0.999383096854
0.996298581123
0.997532387415
0.997532387415
0.998766193708
0.999383096854
0.999383096854
0.996915484269
0.998149290561
0.998766193708


In [9]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    scores = cross_val_score(RandomForestClassifier(), X, Y, scoring='accuracy', cv=10)
    predicted = cross_val_predict(RandomForestClassifier(), X, Y, cv=10)
    print(scores.mean(), metrics.accuracy_score(Y, predicted))

0.997531777565 0.998271818294
0.997407407407 0.99703740279
0.991975917553 0.989013702012
0.997160950511 0.99703740279
0.996790427912 0.996913961239
0.997160646055 0.996667078139
0.998148452604 0.998024935193
0.995802621364 0.996420195038
0.997901234568 0.996173311937
0.987160646055 0.987655844957
0.996174057329 0.996790519689
0.996052427273 0.992593506974
